# 校准 $\pi$ 脉冲
*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要
本教程介绍如何通过改变驱动脉冲的振幅来校准 $\pi$ 脉冲。本教程的大纲如下：

- 背景介绍
- 准备工作
- 定义系统哈密顿量
- 振幅扫描
- 余弦拟合
- 总结

## 背景介绍

校准 $\pi$ 脉冲是量子计算中最基本的操作之一。这是因为要实现 X 门，我们需要在 X 通道上输入一个 $\pi$ 脉冲。此外，校准 $\pi$ 脉冲在校准实际硬件方面也起着重要作用。因此，本教程将演示如何使用量脉校准 $\pi$ 脉冲。

## 准备工作

成功安装量脉后，您可以按照本教程运行下面的量脉程序。在运行此教程前，您需要从 Quanlse 和其它常用的 Python 库导入以下包：

In [ ]:
# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam 

# Import related packages
from Quanlse.QOperator import duff, driveX
from Quanlse.QWaveform import gaussian, QJob, QJobList

# Import simulator interface for Quanlse Cloud Service
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import numpy
from numpy import linspace, pi, dot, array, cos

# Import matplotlib
import matplotlib.pyplot as plt

# Import curve_fit function from scipy
from scipy.optimize import curve_fit

## 定义系统哈密顿量

在量子控制领域，我们经常用哈密顿量来描述一个量子系统。一般来说，系统哈密顿量由含时和不含时项组成：

$$
\hat{H}_{\rm total}(t) = \hat{H}_{\rm drift} + \hat{H}_{\rm ctrl }(t) .
$$

我们从一个具有三能级的单量子比特系统开始，该系统的哈密顿量可以写成：

$$
\hat{H} = \alpha_q \hat{a}^{\dagger}\hat{a}^{\dagger}\hat{a}\hat{a} + \frac{1}{2} c(t) \cos(\phi) (\hat{a}+\hat{a}^{\dagger}).
$$

这里，$\alpha_q$ 是量子比特第 0 到第 1 能级和第 1 到第 2 能级的跃迁能量之间的失谐性；$c(t)$ 表示脉冲包络函数；而 $\phi$ 是脉冲相位。$\hat{a}^{\dagger}=|1\rangle\langle 0|+\sqrt{2}|2\rangle\langle 1|$ 和 $\hat{a}=|0\rangle\langle 1|+\sqrt{2}|1\rangle\langle 2|$ 分别是产生和湮灭算符。

用户可以使用量脉中的 `QHamiltonian` 模块方便地定义多量子比特系统的哈密顿量。接下来，我们将演示如何使用量脉定义上面的哈密顿量。首先，我们初始化系统哈密顿量：

In [ ]:
ham = QHam(subSysNum=1, sysLevel=3, dt=0.2)

上面的 `QHam()` 函数返回一个初始化的哈密顿量，它的参数包括量子比特的数量及其能级数量，采样频率等。

然后我们可以利用 `addDrift()` 函数将漂移项的算符添加到哈密顿量中，该函数的参数中包括一个 `QHam` 对象 `ham`、相应的算符（我们在 `QOperator` 中包含了许多常用的算符）、算符所作用的量子位索引以及振幅系数 `coef`：

In [ ]:
alphaQ = - 0.22 * (2 * pi)  # unit is GHz
ham.addDrift(duff(3), 0, coef=alphaQ)

这里，我们使用 `QOperator` 模块中的 `duff(n)` 函数来定义 $n$ 维的 $\hat{a}^{\dagger}\hat{a}^{\dagger}\hat{a}\hat{a}$；然后，用户可以使用 `print()` 函数来显示哈密顿量的属性：

In [ ]:
print(ham)

接下来，我们通过 `addWave()` 将控制项添加到先前我们定义好的哈密顿量中。与之前版本的量脉相比，我们更新了通过同时添加算符及其相应波形来添加控制脉冲的策略。在这里，我们需要加脉冲项：

$$
c(t) = A e^{-(t-\tau)^2/2\sigma^2}.
$$

我们需要向 `addWave()` 输入控制项算符 `driveX()`，目标量子位索引及其波形（量脉支持多个波形的定义）以及所定义波形所需的参数： 

In [ ]:
ham.addWave(driveX(3), 0, gaussian(t0=0, t=20, a=1.0, tau=10.0, sigma=3.0))

目前为止，我们定义了一个完整的量子系统和控制该系统的相关参数。我们可以使用 `plot()` 来可视化哈密顿量中 `QJob` 的脉冲任务。该函数还包括一个可选的 bool 参数 `dark`，用于启用暗色模式。此外，用户还可以使用 `color` 参数为脉冲指定颜色（如果脉冲数多于颜色数，则颜色将重复使用）。

In [ ]:
ham.job.plot(dark=True, color=['mint'])

然后我们可以使用 `simulate()` 函数来模拟相应系统的演化，并获得系统的酉矩阵的时间演化：

In [ ]:
result = ham.simulate(recordEvolution=False)
result.result

## 脉冲扫描


在脉冲时间 $t_g$ 固定的情况下，我们可以扫描脉冲的振幅 $a$，找到相应 $\pi$ 脉冲的振幅 $a_{\pi}$ 。


我们首先创建一个含有 200 个点的等间距的列表（ -1 到 1 之间），用于表示脉冲的振幅。

In [ ]:
# Initilize the pulse's amplitudes
aList = linspace(-1.0, 1.0, 200)

然后，通过模拟上一节我们定义的哈密顿量的时间演化，我们可以得到每个态的时间演化。在本地设备上进行酉矩阵的演化运算通常需要很长时间，但是我们可以通过量脉云服务来加快这一过程。要使用量脉云服务，用户需要从 http://quantum-hub.baidu.com 获取一个 token 从而将任务提交到量脉的服务器。量脉支持批量任务的提交处理，这可以进一步优化资源的分配。

In [ ]:
# Calibrate a Pi Pulse
jobList = ham.createJobList()
for a in aList:
    # Configure pulse parameters
    job = jobList.createJob()
    job.addWave(driveX(3), 0, gaussian(0, 20, a=a, tau=10, sigma=3))
    jobList.addJob(job)

# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

# Submit batch jobs to Quanlse Cloud Service
resultList = runHamiltonian(ham, jobList=jobList)

# Calculate populations
pop0List = []
pop1List = []
pop2List = []
for result in resultList:
    finalState = dot(result["unitary"], array([1, 0, 0], dtype=complex))
    pop0List.append(abs(finalState[0])**2)
    pop1List.append(abs(finalState[1])**2)
    pop2List.append(abs(finalState[2])**2)

# Plot graph
plt.plot(aList, pop0List, label="Ground state")
plt.plot(aList, pop1List, label="1st excited state")
plt.plot(aList, pop2List, label="2nd excited state")
plt.xlabel("Amplitude")
plt.ylabel("Population of different states")
plt.legend()
plt.show()

## 余弦拟合

现在我们获得了一组离散的点，要找到 $\pi$ 脉冲的振幅，我们需要用余弦函数拟合这些点。为了拟合 $|0\rangle$ 态的时间演化，我们使用 `Scipy` 中的 `optimize.curve_fit()` 函数。我们首先定义以下函数：

In [ ]:
def fit_function(xValues, yValues, initParams):
    def fit_func(x, A, B, period, phi):
        return A * cos(2 * pi * x / period - phi) + B
    fitParams, _ = curve_fit(fit_func, xValues, yValues, initParams, bounds=(0, [2.0, 2.0, 2.0, 2.0]), method='dogbox')
    yFit = fit_func(xValues, *fitParams)
    return fitParams, yFit


然后我们使用回归函数得到结果：

In [ ]:
fitParams, yFit = fit_function(aList, pop0List, [0.5, 0.5, 0.8, 0])

# Plot graph
plt.scatter(aList, pop0List, label="Samples")
plt.plot(aList, yFit, color="red", label="Fit curve")
plt.xlabel("Amplitude")
plt.ylabel("Population of ground state")
plt.legend()
plt.show()
print(f"Period is {fitParams[2]}")
print(f"Pi pulse amplitude is {fitParams[2] / 2}")

通过余弦回归，我们确定了 $\pi$ 脉冲的相应振幅约为0.42。

## 总结

在阅读完这篇有关校准 $\pi$ 脉冲的教程后，我们鼓励用户尝试不同于本教程的参数值以更好地理解如何使用量脉。